In  a docker terminal run
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root

then open jupyter in the printed link (such as http://127.0.0.1:8888/tree?token=4d9d578397aede50fc5bd2d92794b562a6bbcbc73b2dfe51)

CODE HERE, REFRESH AND EXECUTE IN THE BROWSER

In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


In [2]:
sys.path.append('/app')

os.environ['DJANGO_SETTINGS_MODULE'] = 'app.settings'
os.environ['PYTHONPATH'] = '/app'

django.setup()

In [3]:
from toolbox import models
from django.contrib.gis.geos import Point, Polygon, GEOSGeometry
import pandas as pd
import os

In [48]:
csv = pd.read_csv('/app/toolbox/TUdata_import/LfU/Strausberg_faehre/messreihen.csv', delimiter=';')
csv

,Datum,Prüfix,Wasserstand [m],Hinweise
0,01.11.1970,NaN,"65,39",Geprüft
1,02.11.1970,NaN,"65,39",Geprüft
2,03.11.1970,NaN,"65,41",Geprüft
3,04.11.1970,NaN,"65,41",Geprüft
4,05.11.1970,NaN,"65,41",Geprüft
...,...,...,...,...
19755,31.01.2025,NaN,"64,21",Ungeprüft
19756,01.02.2025,NaN,"64,21",Ungeprüft
19757,02.02.2025,NaN,"64,21",Ungeprüft
19758,03.02.2025,NaN,"64,21",Ungeprüft


In [139]:
type(csv.iloc[:1, 0][0])

str

In [140]:
absolute_elevation_of_sensor_m = float(pd.unique(csv.iloc[:, 3])[0].replace(',','.'))
absolute_elevation_of_sensor_m

39.588

In [45]:
pt = Point(424066, 5825789, srid=25833)

In [46]:
## create the Waterlevel location
station = models.Station(
    name='Strausberg, Fähre',
    geom=pt,
    data_provider='LfU Brandenburg',
    absolute_elevation_of_sensor_m=64.134)

In [47]:
station.save()

In [49]:
#csv = csv.drop(csv.columns[3], axis=1)
csv

,Datum,Prüfix,Wasserstand [m],Hinweise
0,01.11.1970,NaN,"65,39",Geprüft
1,02.11.1970,NaN,"65,39",Geprüft
2,03.11.1970,NaN,"65,41",Geprüft
3,04.11.1970,NaN,"65,41",Geprüft
4,05.11.1970,NaN,"65,41",Geprüft
...,...,...,...,...
19755,31.01.2025,NaN,"64,21",Ungeprüft
19756,01.02.2025,NaN,"64,21",Ungeprüft
19757,02.02.2025,NaN,"64,21",Ungeprüft
19758,03.02.2025,NaN,"64,21",Ungeprüft


In [50]:
#date
csv.iloc[:, 0] = pd.to_datetime(csv.iloc[:, 0], format='%d.%m.%Y')


In [51]:
#csv.iloc[:, 1] = csv.iloc[:, 1].str.replace(',', '.').astype(float) # rainfall
csv.iloc[:, 2] = csv.iloc[:, 2].str.replace(',', '.').astype(float) # waterlevel
#csv.iloc[:, 3] = csv.iloc[:, 3].str.replace(',', '.').astype(float) # abs waterlevel


In [52]:
csv

,Datum,Prüfix,Wasserstand [m],Hinweise
0,1970-11-01 00:00:00,NaN,65.39,Geprüft
1,1970-11-02 00:00:00,NaN,65.39,Geprüft
2,1970-11-03 00:00:00,NaN,65.41,Geprüft
3,1970-11-04 00:00:00,NaN,65.41,Geprüft
4,1970-11-05 00:00:00,NaN,65.41,Geprüft
...,...,...,...,...
19755,2025-01-31 00:00:00,NaN,64.21,Ungeprüft
19756,2025-02-01 00:00:00,NaN,64.21,Ungeprüft
19757,2025-02-02 00:00:00,NaN,64.21,Ungeprüft
19758,2025-02-03 00:00:00,NaN,64.21,Ungeprüft


In [ ]:
records = [
    models.TimeseriesValues(
        station=station,
        date=row[0],
        #rainfall=row[1],
        #waterlevel_above_sensor_cm=row[2],
        absolute_water_level_elevation_m=row[2]
    ).save()
    for row in csv.itertuples(index=False)
]

In [15]:
recs = models.TimeseriesValues.objects.all()

In [16]:
recs.count()

26481